# plotly_stream

**A class for easily generating streaming plots with plotly**

## The imports

In [ ]:
import plotly_stream as plyst
import plotly.tools as plyt
import plotly.plotly as ply

## Signing in to your Plotly account

The first time you use `plotly_stream` (or the plotly API directly) on a computer, you have to set your credentials. 

In [ ]:
import json
pc = json.load(open('creds/plotly_creds.json', 'r'))

In [ ]:
plyt.set_credentials_file(username=pc['username'], api_key=pc['api_key'])

where `username` and `api_key` can be found on the settings page of your Plotly account. 
The command creates a credentials file `~/.plotly/.credentials`. 

To stream to Plotly you also need **streaming tokens** which you can create on the settings page of your Plotly account. 

Unfortunately, and in contrary to Plotly's documentation, `set_credentials_file` does not write the streaming tokens to the credentials file, so we have to do that on our own:

In [ ]:
plyst.plotly_stream.set_stream_tokens(pc['stream_ids'])

## A first streaming example

**Note:** At the moment, streaming runs in a blocking way, so you will have to interrupt the kernel after any example in order to proceed. 

The **plotly_stream** constructor has the following syntax:

    ploty_stream(title, subplot_rows, subplot_columns, maxpoints, token_number)
    
Here:
- **title:**           Is the title of the whole plot.
- **subplot_rows:**     Describes the number of subplot rows, default is 1.
- **subplot_columns:** Describes the number of subplot columns, default is 1.
- **maxpoints:**       The maximum number of points
displayed by each trace of the plot, default is 50.
- **token_number:**    By default, any instance of *plotly_stream* takes the first (and subsequent if necessary) streaming tokens from the tokens list. To avoid that a token is used twice, *token_number* can set to another starting point within the tokens list.

In [ ]:
plot = plyst.plotly_stream('A first streaming plot')

The first thing we need is a data source. To this end, we use

       add_data_source(ip, port)

For this tutorial, The Python Quants have set up 3 dummy data servers which run under the IP 80.82.223.74 on ports 6669 to 6671.

In [ ]:
sock = plot.add_data_source('80.82.223.74', 6669)

Next, we can define a trace object by using the method **add_trace**. The syntax is

    add_trace(kind, data_source, name, row, col, parse_data)
    
Where 
 - **kind** is the kind of plot, one of '*scatter*' or '*bars*'.
 - **data_source** is a data source as defined above.
 - **name** is the name of the trace as it appears in the plot's legend for example; default is *None*.
 - **row** and **col** define the subplot the trace should appear in; default is 1 for both parameters.
 - **parse_data** is an optional callback function to parse the incoming data; the function must accept one string parameter (the incoming message from the data source) and return the new x and y values for the plot.

In [ ]:
plot.add_trace('scatter', sock, 'Data from Server 1')

In order to embed the plot to this notebook, we need the plot url. 

In [ ]:
url = plot.get_plot_url()
plyt.embed(url)

After that, we can start our first stream with 

    start_streaming(auto_open)
    
where **auto_open** is a boolean. If True, which is default, the browser opens automatically the new plot.

If there was no former call of ***plyt.embed(self.plot_url)*** the method prints the url of the new plot.

In [ ]:
plot.start_streaming(auto_open=False)

To preceed, interrupt the kernel. To restart the streaming you can always execute the cell above again.

## Working with subplots and layout

In this example, we will create a plot with 3 subplots with some additional layout. We also introduce the ***parse_data*** callback.

In [ ]:
plot = plyst.plotly_stream('Two Subplots', subplot_rows=1, subplot_columns=2)
sock = plot.add_data_source('80.82.223.74', 6669)

In [ ]:
plot.add_trace('scatter', sock, 'Data from server 1', row=1, col=1)

So far, nothing new. But for the second plot, we want the x axis values as number of seconds since midnight and not as time. For that reason, we write a callback function:

In [ ]:
def time_to_seconds(msg):
    # input is of form "time, value", where time is hh:mm:ss.msmsms
    parts = msg.split(',')
    x1 = str(parts[0])
    x = int(x1[:2]) * 60 * 60 + int(x1[3:5]) * 60 + int(x1[6:8]) + 0.001 * int(x1[10:])
    y = float(parts[1])
    return x, y

The new trace is defined with that callback function.

In [ ]:
plot.add_trace('scatter', sock, 'Data from server 1 in seconds', 1, 2, time_to_seconds)

Defining titles for the subplots is easy:

In [ ]:
plot.set_subplot_title(1, 1, 'Data from Server 1 unparsed')
plot.set_subplot_title(1, 2, 'Data from Server 1 parsed')

Adding layout features is accomplished by with ***add_layout***. The syntax is

    add_layout(target, values)
    
where
 - **target** is the object on which the layout is applied, examples are `yaxis, xaxis, legend`; if target is empty, the layout is applied to the whole plot
 - **values** is a dictionary with the layout keys/values

Setting the width and height of the plot:

In [ ]:
plot.add_layout('', {'height': 1000, 'width': 600})

Setting labels and grids for the axes:

In [ ]:
plot.add_layout('yaxis1', {'title': 'Y-Axis 1', 'showgrid': True})
plot.add_layout('yaxis2', {'title': 'Y-Axis 2', 'showgrid': False})

plot.add_layout('xaxis1', {'title': 'As times', 'showgrid': True})
plot.add_layout('xaxis2', {'title': 'As seconds', 'showgrid': False})

Setting ticks on the axes:

In [ ]:
y_ticks = dict(autotick=False,
               ticks='outside',
               tick0=0,
               dtick=10,
               ticklen=8,
               tickwidth=2,
               tickcolor='#000'
              )
plot.add_layout('yaxis1', y_ticks)

x_ticks = dict(tickfont=dict(
                   size=10,
                   color='blue'
                   ),
               tickangle=45)

plot.add_layout('xaxis1', x_ticks)

Let's have a look at what we have defined so far ...

In [ ]:
url = plot.get_plot_url()
plyt.embed(url)

... and start the streaming.

In [ ]:
plot.start_streaming(auto_open=False)

## Several traces in one plot

In this example, we add a new row of subplots to the figure above, containing one plot spanning both columns with the streamed data of both server 1 and server 2.

In [ ]:
plot = plyst.plotly_stream('Streaming several traces in one plot', subplot_rows=2, subplot_columns=2)
sock = plot.add_data_source('80.82.223.74', 6669)
sock2 = plot.add_data_source('80.82.223.74', 6670)

plot.add_trace('scatter', sock, 'Data from server 1', row=1, col=1)
plot.add_trace('scatter', sock2, 'Data from server 2', row=1, col=2)

plot.set_subplot_title(1, 1, 'Data from Server 1')
plot.set_subplot_title(1, 2, 'Data from Server 1')

The next two traces will both be inserted into subplot 2/1.

In [ ]:
plot.add_trace('scatter', sock, 'Data from server 1', 2, 1)
plot.add_trace('scatter', sock2, 'Data from server 2', 2, 1)

plot.set_subplot_title(2, 1, 'Combined Data from Server 1 & 2')

To combine subplot 2/1 and subplot 2/2 we use the method 

    set_subplot_spec()    

In [ ]:
plot.set_subplot_specs(2, 1, {'colspan': 2})
plot.set_subplot_specs(2, 2, None)

In [ ]:
x_ticks = dict(tickfont=dict(
                   size=10,
                   ),
               tickangle=45)

plot.add_layout('xaxis1', x_ticks)
plot.add_layout('xaxis2', x_ticks)
plot.add_layout('xaxis3', x_ticks)

In [ ]:
url = plot.get_plot_url()
plyt.embed(url)

In [ ]:
plot.start_streaming(auto_open=False)

## Ploting bars

Plotting bars is analogous to scatter plots, the only difference being that the optional callback function must return two list object, namely for the the bars labels and the bars values.

In [ ]:
plot = plyst.plotly_stream('CPU usage of PythonQuants01')
sock3 = plot.add_data_source('80.82.223.74', 6671)

def callback_for_bars(msg):
    parts = msg.split(',')
    y = [float(p)+1 if float(p)==0 else float(p) for p in parts]
    x = ['cpu %s' %(i+1) for i in range(len(parts))]
    return x, y

plot.add_trace('bars', sock3, 'cpu usage', parse_data=callback_for_bars )
plot.add_layout('yaxis1', {'range': [0,100]})
plot.add_layout('yaxis1', {'title': 'cpu usage in %'})

In [ ]:
url = plot.get_plot_url()
plyt.embed(url)

In [ ]:
plot.start_streaming(auto_open=False)